In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException
import os
import time
import pandas as pd

In [7]:
pd.read_csv('equipment_data.csv').to_excel('equipment_data.xlsx')

In [159]:
service = Service("C://Users//servi//inter-scraping//chromedriver-win64//chromedriver-win64//chromedriver.exe")
chrome_profile_path = r'C:\Users\servi\AppData\Local\Google\Chrome\User Data\Default'

# Set up Chrome options
# Configuração das opções do Chrome
options = webdriver.ChromeOptions()
options.add_argument(f"user-data-dir={chrome_profile_path}")
options.add_experimental_option("detach", True)  # Isso permite que o Chrome permaneça aberto após o script terminar

# Inicializa o Chrome com as opções
driver = webdriver.Chrome(service=service)
"""
https://gestao.granitopagamentos.com.br/Login/Index
sandro.leao@creditoessencial.com.br
11S@ndr01003
"""

'\nhttps://gestao.granitopagamentos.com.br/Login/Index\nsandro.leao@creditoessencial.com.br\n11S@ndr01003\n'

In [5]:
linhas = driver.find_elements(By.XPATH, "//table[@id='tabelaClientes']/tbody/tr")


In [9]:
linhas[0].click()

In [167]:
def change_window(window_name):
    link_pesquisa = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.LINK_TEXT, window_name))
    )
    try:
        link_pesquisa.click()
    except ElementClickInterceptedException:
        time.sleep(0.5)
    wait_for_processing_to_disappear(driver)


In [13]:
change_window()

In [29]:

equipments_rows = driver.find_elements(By.XPATH, '//*[@id="tabelaPDV"]')
for row in equipments_rows:
    serie_number = row.find_element(By.XPATH, ".//td[2]").text
    pdv = row.find_element(By.XPATH, ".//td[1]").text

In [23]:
linha = equipments_rows[0]

In [93]:
pdv = equipment_row.find_element(By.XPATH, ".//td[0]").text


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//td[0]"}
  (Session info: chrome=134.0.6998.35); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF65BAB6EE5+28773]
	(No symbol) [0x00007FF65BA225D0]
	(No symbol) [0x00007FF65B8B8FAA]
	(No symbol) [0x00007FF65B90F286]
	(No symbol) [0x00007FF65B90F4BC]
	(No symbol) [0x00007FF65B901C6C]
	(No symbol) [0x00007FF65B93728F]
	(No symbol) [0x00007FF65B901B36]
	(No symbol) [0x00007FF65B937460]
	(No symbol) [0x00007FF65B95F6F3]
	(No symbol) [0x00007FF65B937023]
	(No symbol) [0x00007FF65B8FFF5E]
	(No symbol) [0x00007FF65B9011E3]
	GetHandleVerifier [0x00007FF65BE0422D+3490733]
	GetHandleVerifier [0x00007FF65BE1BA13+3586963]
	GetHandleVerifier [0x00007FF65BE1144D+3544525]
	GetHandleVerifier [0x00007FF65BB7C9AA+838442]
	(No symbol) [0x00007FF65BA2D01F]
	(No sy

In [171]:
equipment_data = {
    'name': [], 'cpf_cnpj': [],
    'equipment_id': [], 'pdv': []
}

In [173]:
def wait_for_processing_to_disappear(driver, timeout=10):
    """ Aguarda a aba 'Processando...' desaparecer antes de continuar """
    try:
        WebDriverWait(driver, timeout).until(
            EC.invisibility_of_element_located((By.XPATH, "//*[contains(text(), 'Processando')]"))
        )
        print("Aba 'Processando...' desapareceu. Continuando...")
    except Exception as e:
        print(f"Erro ou tempo esgotado ao esperar a aba 'Processando...': {e}")

In [177]:
page = 1
while True:
    client_rows = driver.find_elements(By.XPATH, "//table[@id='tabelaClientes']/tbody/tr")
    for i, client_row in enumerate(client_rows):
        try:
            cpf_cnpj = client_row.find_element(By.XPATH, ".//td[3]").text
            name = client_row.find_element(By.XPATH, ".//td[4]").text
            print(name, cpf_cnpj)
            print(f'{i+1}/{len(client_rows)} da pag. {page}')

            client_row.click()
            change_window('Equipamentos')
            time.sleep(1)

            # Espera até que os elementos estejam disponíveis
            max_retries = 5
            retries = 0
            while retries < max_retries:
                try:
                    equipment_rows = driver.find_elements(By.XPATH, "//table[@id='tabelaPDV']/tbody/tr")
                    
                    if not equipment_rows:
                        break  # Nenhum equipamento encontrado, segue para o próximo cliente

                    for equipment_row in equipment_rows:
                        equipment_id = equipment_row.find_element(By.XPATH, ".//td[2]").text
                        pdv = equipment_row.find_element(By.XPATH, ".//td[1]").text
                        equipment_data['name'].append(name)
                        equipment_data['cpf_cnpj'].append(cpf_cnpj)
                        equipment_data['equipment_id'].append(equipment_id)
                        equipment_data['pdv'].append(pdv)

                    break  # Sai do `while` se tudo der certo

                except StaleElementReferenceException:
                    retries += 1
                    time.sleep(1)  # Tenta novamente após um pequeno delay

            change_window('Pesquisa')
            time.sleep(1)

        except Exception as e:
            print(f"Erro ao processar cliente {i+1}: {e}")
            continue  # Continua para o próximo cliente, evitando travamento

    # Verifica se o botão "Próximo" está desativado
    try:
        next_button_li = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "tabelaClientes_next"))
        )
        
        if 'disabled' in next_button_li.get_attribute('class'):
            print("Fim da paginação. Encerrando.")
            break
            
        next_button = next_button_li.find_element(By.TAG_NAME, "a")
        page += 1
        next_button.click()

    except Exception as e:
        print(f"Erro ao clicar em 'Próximo': {e}")
        break  # Sai do loop se houver erro na paginação

Victor Emannuel Medeiros Silva 017.841.496-40
1/10
Aba 'Processando...' desapareceu. Continuando...
Aba 'Processando...' desapareceu. Continuando...
Euires Teixeira de Melo 977.615.086-15
2/10
Aba 'Processando...' desapareceu. Continuando...
Aba 'Processando...' desapareceu. Continuando...
Francisco José da Silva 029.913.726-05
3/10
Aba 'Processando...' desapareceu. Continuando...
Aba 'Processando...' desapareceu. Continuando...
54.112.002 AYARA ANTONIETA BAZILIO BARBOSA 54.112.002/0001-00
4/10
Aba 'Processando...' desapareceu. Continuando...
Aba 'Processando...' desapareceu. Continuando...
Raquel Bazilio Corrêa Borges 009.543.786-01
5/10
Aba 'Processando...' desapareceu. Continuando...
Aba 'Processando...' desapareceu. Continuando...
Santa Pereira de Oliveira Gomes 768.362.316-49
6/10
Aba 'Processando...' desapareceu. Continuando...
Aba 'Processando...' desapareceu. Continuando...
Nilton Nunes da Silva 863.943.046-20
7/10
Aba 'Processando...' desapareceu. Continuando...
Aba 'Processan

In [185]:
df = pd.DataFrame(equipment_data)
df.to_csv('equipment_data.csv')

In [99]:
import pandas as pd
pd.DataFrame(equipment_data)

,name,cpf_cnpj,equipment_id,pdv
0,Victor Emannuel Medeiros Silva,017.841.496-40,JA8300482382,1
1,Euires Teixeira de Melo,977.615.086-15,,1
2,Francisco José da Silva,029.913.726-05,,1
3,54.112.002 AYARA ANTONIETA BAZILIO BARBOSA,54.112.002/0001-00,JA6600033830,1
4,Raquel Bazilio Corrêa Borges,009.543.786-01,,1
5,Santa Pereira de Oliveira Gomes,768.362.316-49,,1
6,Nilton Nunes da Silva,863.943.046-20,JA8400488664,1
7,CARLOS AUGUSTO GOMES,073.905.896-71,JA8400488664,1
8,Marcela Carolina Ribeiro Menezes,067.316.626-07,,1
9,Paula Miranda Moraes,069.116.786-92,JA7A00185378,1


In [113]:
equipment_rows = driver.find_elements(By.XPATH, "//table[@id='tabelaPDV']/tbody/tr")
for equipment_row in equipment_rows:
    equipment_id = equipment_row.find_element(By.XPATH, ".//td[2]").text
    pdv = equipment_row.find_element(By.XPATH, ".//td[1]").text
    equipment_data['name'].append(name)
    equipment_data['cpf_cnpj'].append(cpf_cnpj)
    equipment_data['equipment_id'].append(equipment_id)
    equipment_data['pdv'].append(pdv)
    print(equipment_id)

JA8300482382
JA8400507804


In [107]:
len(equipment_rows)

1

In [57]:
name = linha.find_element(By.XPATH, ".//td[4]").text


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//td[4]"}
  (Session info: chrome=134.0.6998.35); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF65BAB6EE5+28773]
	(No symbol) [0x00007FF65BA225D0]
	(No symbol) [0x00007FF65B8B8FAA]
	(No symbol) [0x00007FF65B90F286]
	(No symbol) [0x00007FF65B90F4BC]
	(No symbol) [0x00007FF65B901C6C]
	(No symbol) [0x00007FF65B93728F]
	(No symbol) [0x00007FF65B901B36]
	(No symbol) [0x00007FF65B937460]
	(No symbol) [0x00007FF65B95F6F3]
	(No symbol) [0x00007FF65B937023]
	(No symbol) [0x00007FF65B8FFF5E]
	(No symbol) [0x00007FF65B9011E3]
	GetHandleVerifier [0x00007FF65BE0422D+3490733]
	GetHandleVerifier [0x00007FF65BE1BA13+3586963]
	GetHandleVerifier [0x00007FF65BE1144D+3544525]
	GetHandleVerifier [0x00007FF65BB7C9AA+838442]
	(No symbol) [0x00007FF65BA2D01F]
	(No sy